https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/05-deployment/workshop

In this homework, we're going to continue working with the lead scoring dataset.

### Question 1

In [1]:
!uv --version

uv 0.9.7


Ans: 0.9.7

In [2]:
# Initialize an empty uv project
# uv init

### Question 2

* Use uv to install Scikit-Learn version 1.6.1
* What's the first hash for Scikit-Learn you get in the lock file?
* Include the entire string starting with sha256:, don't include quotes

In [6]:
!uv add scikit-learn==1.6.1 

Using CPython 3.12.1 interpreter at: /home/codespace/.python/current/bin/python3.12
Creating virtual environment at: .venv
Resolved 6 packages in 93ms                                          
Prepared 5 packages in 925ms                                             
░░░░░░░░░░░░░░░░░░░░ [0/5] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 5 packages in 262ms                               
 + joblib==1.5.2
 + numpy==2.3.4
 + scikit-learn==1.6.1
 + scipy==1.16.3
 + threadpoolctl==3.6.0


Ans: sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e

### Models

We have prepared a pipeline with a dictionary vectorizer and a model. And then saved with Pickle

In [3]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

--2025-11-01 12:30:09--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-11-01 12:30:09--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin’

pipeline_v1.bin

### Question 3

Let's use the model!
Write a script for loading the pipeline with pickle

In [4]:
import pickle

In [5]:
# Score this record:
record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

In [7]:
with open('pipeline_v1.bin', 'rb') as f_in:
    pipeline = pickle.load(f_in)

In [9]:
# What's the probability that this lead will convert?
pipeline.predict_proba(record)[0,1]

np.float64(0.5336072702798061)

Ans: 0.533

### Question 4

Now let's serve this model as a web service:
* Install FastAPI
* Write FastAPI code for serving the model

In [12]:
!uv add fastapi uvicorn

Resolved 116 packages in 380ms                                       
Prepared 2 packages in 12ms                                              
░░░░░░░░░░░░░░░░░░░░ [0/2] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 2 packages in 3ms                                 
 + click==8.3.0
 + uvicorn==0.38.0


In [17]:
import requests

In [26]:
url = "http://localhost:9696/predict"

client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

requests.post(url, json=client).json()

{'subscription_probability': 0.5340417283801275, 'subscription': True}

What's the probability that this client will get a subscription?

Ans: 0.534

### Docker

For these questions, we prepared a base image: `agrigorev/zoomcamp-model:2025`. You'll need to use it

This image is based on `3.13.5-slim-bookworm` and has a pipeline with logistic regression (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

### Question 5

Download the base image `agrigorev/zoomcamp-model:2025`

In [32]:
!docker pull agrigorev/zoomcamp-model:2025

2025: Pulling from agrigorev/zoomcamp-model

2667830b: Pulling fs layer 
1082f6f0: Pulling fs layer 
f5177ae2: Pulling fs layer 
357cf7a1: Pulling fs layer 
2b9cc6ee: Pulling fs layer 
Digest: sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae
Status: Downloaded newer image for agrigorev/zoomcamp-model:2025
docker.io/agrigorev/zoomcamp-model:2025


In [33]:
# what's the size of this base image?
!docker images

REPOSITORY                 TAG       IMAGE ID       CREATED       SIZE
agrigorev/zoomcamp-model   2025      4a9ecc576ae9   11 days ago   121MB


Ans: 121 MB

### Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

Now complete it:

* Install all the dependencies from pyproject.toml
* Copy your FastAPI script
* Run it with uvicorn

In [37]:
# docker build -t subscription-prediction .
# docker run -it --rm -p 9696:9696 -e MODEL_NAME=pipeline_v2.bin subscription-prediction

### Question 6

After running it, score this client once again

In [38]:
requests.post(url, json=client).json()

{'model_name': 'pipeline_v2.bin',
 'subscription_probability': 0.9933071490756734,
 'subscription': True}

Ans: 0.99